In [4]:
!pip install flask
!pip install sympy
!pip install flask_cors
!pip install sympy
!pip install mysql-connector-python
!pip install mediapipe opencv-python

from urllib import parse
from unicodedata import category
from flask import Flask,request,redirect,render_template
from flask_cors import CORS,cross_origin
from fractions import Fraction
from sympy import re

#--------------------------------------------------
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils

mp_pose = mp.solutions.pose
import pandas as pd
from PIL import ImageFont, ImageDraw, Image
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import math as m
#---------------------------------------------------------
import mysql.connector


from ast import AsyncFunctionDef
from audioop import cross
from unittest import result
from enum import auto
from multiprocessing.sharedctypes import Value
from turtle import width
from urllib import parse

In [5]:


def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

def KneeDistance(x1, y1, x2, y2):

    dist = round(m.sqrt((x2-x1)**2+(y2-y1)**2),4)

    return dist

def Angle(x1, y1, x2, y2, x3, y3):
    
    if x1 == x2 or x2 == x3 :
        degree = 90
    elif y1 == y2 or y2 == y3:
        degree = 0
    else :
        theta = m.atan( (y1- y2) / ( x1- x2)) - m.atan( (y3 - y2) / (x3 - x2))
        degree = int(180/m.pi)*theta

    
    if degree <0 :
        degree = degree*(-1)
    return degree 

# 랫풀다운 함수 모음 (트레이닝, 챌린지)
#
#

# 랫풀다운 챌린지 insert
def insertLat_CH(nickname, acc):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    
    sql=f"INSERT INTO ranking (nickname, ex_name, rank_acc, rank_date) values ('{nickname}', '랫 풀 다운', {acc}, sysdate())"
    cursor.execute(sql)
    conn.commit()
       
    cursor.close()
    conn.close()

#랫풀다운 트레이닝 insert
def insertLat_TR(val, nickname, acc):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    for i in range(len(val)):
        sql=f"INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('{nickname}', '랫 풀 다운', {acc}, sysdate(), '{val[i]}')"
        cursor.execute(sql)
        conn.commit()
       
    cursor.close()
    conn.close()

# 랫 풀 다운 시작----------------------------------------------------------------------------------------------------------------------------------------------------------------------

def lat_CH(nickname):
    
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
       

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <30 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                    
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                
                        
                elif len(b) < len(a) :


                    if angle > 70:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                    
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
            
                        
                elif len(c) < len(b) :


                    if angle > 160 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
    
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle <70:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
            
                        
                elif len(e) < len(d) :
        

                    if angle <40 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
        
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)

                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



        df = pd.DataFrame([f1,f2, f3, f4, f5])
        df.columns = ["leftElbowAngle0","leftShoulderAngle0","rightElbowAngle0","rightShoulderAngle0",
                "leftElbowAngle5","leftShoulderAngle5","rightElbowAngle5","rightShoulderAngle5",
                "leftElbowAngle8","leftShoulderAngle8","rightElbowAngle8","rightShoulderAngle8",
                "leftElbowAngle12","leftShoulderAngle12","rightElbowAngle12","rightShoulderAngle12",
                "leftElbowAngle15","leftShoulderAngle15","rightElbowAngle15","rightShoulderAngle15"]
        scalar = StandardScaler()
        x_data_scaled = scalar.fit_transform(df)
        model = load_model('latpullpredict1.h5')

        a = np.array(x_data_scaled)
        b = model.predict(a)
        c = np.argmax(b, axis=1)
        
    
    acc=0.0

    for i in range(len(c)):
        if c[i]==0:
            acc+=20
        
           
   
    
    insertLat_CH(nickname, acc)


def lat_TR(nickname):
    
    
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
       

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <30 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                    
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                
                        
                elif len(b) < len(a) :


                    if angle > 70:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                    
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
            
                        
                elif len(c) < len(b) :


                    if angle > 160 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
    
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle <70:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
            
                        
                elif len(e) < len(d) :
        

                    if angle <40 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
        
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)

                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



        df = pd.DataFrame([f1,f2, f3, f4, f5])
        df.columns = ["leftElbowAngle0","leftShoulderAngle0","rightElbowAngle0","rightShoulderAngle0",
                "leftElbowAngle5","leftShoulderAngle5","rightElbowAngle5","rightShoulderAngle5",
                "leftElbowAngle8","leftShoulderAngle8","rightElbowAngle8","rightShoulderAngle8",
                "leftElbowAngle12","leftShoulderAngle12","rightElbowAngle12","rightShoulderAngle12",
                "leftElbowAngle15","leftShoulderAngle15","rightElbowAngle15","rightShoulderAngle15"]
        scalar = StandardScaler()
        x_data_scaled = scalar.fit_transform(df)
        model = load_model('latpullpredict1.h5')

        a = np.array(x_data_scaled)
        b = model.predict(a)
        c = np.argmax(b, axis=1)
        
    val=[]
    acc=0

    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("상체가 불안정하며, 손목이 꺾였습니다")
        elif c[i]==2:
            val.append("상체가 불안정하며, 축 평행이 불량합니다.")
        elif c[i]==3:
            val.append("상체가 불안정하며, 기구 당김이 부족합니다")
   
    print(val)
    
    insertLat_TR(val, nickname, acc)


# 랫풀 다운 함수 모음 끝

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------

#바벨 컬 트레이닝 TR insert
def insertcal_TR(val, nickname, acc):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    print("바벨컬 ")
    
    for i in range(len(val)):
        sql=f"INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('{nickname}', '바벨 컬', {acc}, sysdate(), '{val[i]}')"
        cursor.execute(sql)
        conn.commit()
       
    cursor.close()
    conn.close()


# 바벨 컬 챌린지 CH insert
def insertcal_CH(nickname, acc):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    sql=f"INSERT INTO ranking (nickname, ex_name, rank_acc, rank_date) values ('{nickname}', '바벨 컬', {acc}, sysdate())"
        
    cursor.execute(sql)
    conn.commit()
       
    cursor.close()
    conn.close()


# 바벨 컬 시작----------------------------------------------------------------------------------------------------------------------------------------------------------------------


def cal_TR(nickname):

    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                leftWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
                rightWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                        a.append(leftWaistAngle)
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                        a.append(rightWaistAngle)

                        
                elif len(b) < len(a) :


                    if angle < 90:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                        b.append(leftWaistAngle)
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
                        b.append(rightWaistAngle)
                        
                elif len(c) < len(b) :


                    if angle < 30 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                        c.append(leftWaistAngle)
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
                        c.append(rightWaistAngle)
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle >90:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                        d.append(leftWaistAngle)
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
                        d.append(rightWaistAngle)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
                        e.append(leftWaistAngle)
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)
                        e.append(rightWaistAngle)
                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftElbowAngle0", "leftShoulderAngle0", "leftWaistAngle0", "rightElbowAngle0", "rightShoulderAngle0", "rightWaistAngle0",
                "leftElbowAngle5", "leftShoulderAngle5", "leftWaistAngle5", "rightElbowAngle5", "rightShoulderAngle5", "rightWaistAngle5",
                "leftElbowAngle8", "leftShoulderAngle8", "leftWaistAngle8", "rightElbowAngle8", "rightShoulderAngle8", "rightWaistAngle8",
                "leftElbowAngle12", "leftShoulderAngle12", "leftWaistAngle12", "rightElbowAngle12", "rightShoulderAngle12", "rightWaistAngle12",
                "leftElbowAngle15", "leftShoulderAngle15", "leftWaistAngle", "rightElbowAngle15", "rightShoulderAngle15", "rightWaistAngle15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bcpredict.h5')

    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)
    val=[]
    acc=0
    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("팔꿈치가 너무 앞으로 나왔습니다.")
        elif c[i]==2:
            val.append("척추가 말렸으며, 손목이 꺾였습니다.")
        elif c[i]==3:
            val.append("척추가 말렸으며, 운동 속도가 빠릅니다.")
        elif c[i]==4:
            val.append("척추가 말렸으며, 어깨 견착이 불안정하며, 손목이 꺾였습니다.")
        

    
    (val)

    insertcal_TR(val, nickname, acc)


def cal_CH(nickname):

    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                leftElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y)
                
                leftShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                leftWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y)
                
                rightElbowAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y)
                
                rightShoulderAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)
                
                rightWaistAngle = Angle(landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        a.append(leftElbowAngle)
                        a.append(leftShoulderAngle)
                        a.append(leftWaistAngle)
                        a.append(rightElbowAngle)
                        a.append(rightShoulderAngle)
                        a.append(rightWaistAngle)

                        
                elif len(b) < len(a) :


                    if angle < 90:
                        b.append(leftElbowAngle)
                        b.append(leftShoulderAngle)
                        b.append(leftWaistAngle)
                        b.append(rightElbowAngle)
                        b.append(rightShoulderAngle)
                        b.append(rightWaistAngle)
                        
                elif len(c) < len(b) :


                    if angle < 30 :
                        c.append(leftElbowAngle)
                        c.append(leftShoulderAngle)
                        c.append(leftWaistAngle)
                        c.append(rightElbowAngle)
                        c.append(rightShoulderAngle)
                        c.append(rightWaistAngle)
                        stage = 'down'
                        
                elif len(d) < len(c) :

            
                    if angle >90:
                        d.append(leftElbowAngle)
                        d.append(leftShoulderAngle)
                        d.append(leftWaistAngle)
                        d.append(rightElbowAngle)
                        d.append(rightShoulderAngle)
                        d.append(rightWaistAngle)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        e.append(leftElbowAngle)
                        e.append(leftShoulderAngle)
                        e.append(leftWaistAngle)
                        e.append(rightElbowAngle)
                        e.append(rightShoulderAngle)
                        e.append(rightWaistAngle)
                        
                        stage = 'up'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftElbowAngle0", "leftShoulderAngle0", "leftWaistAngle0", "rightElbowAngle0", "rightShoulderAngle0", "rightWaistAngle0",
                "leftElbowAngle5", "leftShoulderAngle5", "leftWaistAngle5", "rightElbowAngle5", "rightShoulderAngle5", "rightWaistAngle5",
                "leftElbowAngle8", "leftShoulderAngle8", "leftWaistAngle8", "rightElbowAngle8", "rightShoulderAngle8", "rightWaistAngle8",
                "leftElbowAngle12", "leftShoulderAngle12", "leftWaistAngle12", "rightElbowAngle12", "rightShoulderAngle12", "rightWaistAngle12",
                "leftElbowAngle15", "leftShoulderAngle15", "leftWaistAngle", "rightElbowAngle15", "rightShoulderAngle15", "rightWaistAngle15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bcpredict.h5')

    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)
   
    acc=0.0
    for i in range(len(c)):
        if c[i]==0:
            acc+=20.0
      
        

   

    insertcal_CH(nickname, acc)

#바벨 컬 끝
#-------------------------------------------------------------------------------------------------------------



# 스쿼트 함수 모음 (트레이닝, 챌린지)

# 스쿼트 챌린지 intsert
def insertSquat_CH(nickname, acc):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    
    sql=f"INSERT INTO ranking (nickname, ex_name, rank_acc, rank_date) values ('{nickname}', '바벨 스쿼트', {acc}, sysdate())"
    cursor.execute(sql)
    conn.commit()
       
    cursor.close()
    conn.close()

# 스쿼트 트레이닝 insert
def insertSquat_TR(nickname, acc, val):
    print("인설트 함수입니다....")
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    for i in range(len(val)):
        sql=f"INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('{nickname}', '바벨 스쿼트', '{acc}', sysdate(), '{val[i]}')"
        cursor.execute(sql)
        conn.commit()
       
    cursor.close()
    conn.close()


#스쿼트 첼린지
def squat_CH(nickname):
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'down'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # Calculate angle
                angle = calculate_angle(hip, knee, ankle)
                
                leftKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
                
                leftHipAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
                
                rightKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y)
                
                rightHipAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
                kneeD = KneeDistance(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(knee, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        print(1)
                        a.append(leftKneeAngle)
                        a.append(leftHipAngle)
                        a.append(rightKneeAngle)
                        a.append(rightHipAngle)
                        a.append(kneeD)
                
                        
                elif len(b) < len(a) :


                    if angle < 140:
                        print(2)
                        b.append(leftKneeAngle)
                        b.append(leftHipAngle)
                        b.append(rightKneeAngle)
                        b.append(rightHipAngle)
                        b.append(kneeD)
                        
                elif len(c) < len(b) :


                    if angle < 95 :
                        print(3)
                        c.append(leftKneeAngle)
                        c.append(leftHipAngle)
                        c.append(rightKneeAngle)
                        c.append(rightHipAngle)
                        c.append(kneeD)
                        stage = 'up'
                        
                elif len(d) < len(c) :

            
                    if angle >140:
                        print(4)
                        d.append(leftKneeAngle)
                        d.append(leftHipAngle)
                        d.append(rightKneeAngle)
                        d.append(rightHipAngle)
                        d.append(kneeD)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        
                        e.append(leftKneeAngle)
                        e.append(leftHipAngle)
                        e.append(rightKneeAngle)
                        e.append(rightHipAngle)
                        e.append(kneeD)
                        print(5)
                        stage = 'down'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftKneeAngle0", "leftHipAngle0", "rightKneeAngle0", "rightHipAngle0", "KneeDistance0",
                "leftKneeAngle5", "leftHipAngle5", "rightKneeAngle5", "rightHipAngle5", "KneeDistance5",
                "leftKneeAngle8", "leftHipAngle8", "rightKneeAngle8", "rightHipAngle8", "KneeDistance8",
                "leftKneeAngle12", "leftHipAngle12", "rightKneeAngle12", "rightHipAngle12", "KneeDistance12",
                "leftKneeAngle15", "leftHipAngle15", "rightKneeAngle15", "rightHipAngle15", "KneeDistance15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bsquatpredict.h5')
    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)
    val=[]
    acc=0.0
    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("척추가 말렸습니다.")
        elif c[i]==2:
            val.append("고개를 내려주세요.")
        elif c[i]==3:
            val.append("무릎을 모아주세요.")

    print(val)
    print(acc)

    insertSquat_TR(nickname, acc, val)

#스쿼트 트레이닝
def squat_TR(nickname):
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'down'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # Calculate angle
                angle = calculate_angle(hip, knee, ankle)
                
                leftKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
                
                leftHipAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
                
                rightKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y)
                
                rightHipAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
                kneeD = KneeDistance(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(knee, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        
                        a.append(leftKneeAngle)
                        a.append(leftHipAngle)
                        a.append(rightKneeAngle)
                        a.append(rightHipAngle)
                        a.append(kneeD)
                
                        
                elif len(b) < len(a) :


                    if angle < 140:
                        
                        b.append(leftKneeAngle)
                        b.append(leftHipAngle)
                        b.append(rightKneeAngle)
                        b.append(rightHipAngle)
                        b.append(kneeD)
                        
                elif len(c) < len(b) :


                    if angle < 95 :
                        
                        c.append(leftKneeAngle)
                        c.append(leftHipAngle)
                        c.append(rightKneeAngle)
                        c.append(rightHipAngle)
                        c.append(kneeD)
                        stage = 'up'
                        
                elif len(d) < len(c) :

            
                    if angle >140:
                        
                        d.append(leftKneeAngle)
                        d.append(leftHipAngle)
                        d.append(rightKneeAngle)
                        d.append(rightHipAngle)
                        d.append(kneeD)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        
                        e.append(leftKneeAngle)
                        e.append(leftHipAngle)
                        e.append(rightKneeAngle)
                        e.append(rightHipAngle)
                        e.append(kneeD)
                        
                        stage = 'down'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftKneeAngle0", "leftHipAngle0", "rightKneeAngle0", "rightHipAngle0", "KneeDistance0",
                "leftKneeAngle5", "leftHipAngle5", "rightKneeAngle5", "rightHipAngle5", "KneeDistance5",
                "leftKneeAngle8", "leftHipAngle8", "rightKneeAngle8", "rightHipAngle8", "KneeDistance8",
                "leftKneeAngle12", "leftHipAngle12", "rightKneeAngle12", "rightHipAngle12", "KneeDistance12",
                "leftKneeAngle15", "leftHipAngle15", "rightKneeAngle15", "rightHipAngle15", "KneeDistance15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bsquatpredict.h5')
    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)

    print("함수 안입니다.")
    val=[]
    acc=0.0
    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("척추가 말렸습니다.")
        elif c[i]==2:
            val.append("고개를 내려주세요.")
        elif c[i]==3:
            val.append("무릎을 모아주세요.")

    insertSquat_TR(nickname, acc, val)


In [6]:
app=Flask(__name__)  #flask 서버 객체 생성

@cross_origin
@app.route("/")

def index():
    url = "https://www.naver.com/"

    return redirect(url)

# 랫 풀 다운 트레이닝 라우더
@cross_origin
@app.route("/lat_TR", methods=["GET", "POST"])
def lat_T():
    if request.method=='POST':
        
        url="http://localhost:8085/flask"
        return redirect(url)
    else:
        nickname=request.args.get('nickname')
        print(nickname)

        lat_TR(nickname)

        ex_name="랫 풀 다운"
        url_2 =f"http://localhost:8085/flaskFeedDetail.do?nickname={nickname}&ex_name={ex_name}"

        return redirect(url_2)


# 랫 풀 다운 챌린지 라우더
@cross_origin
@app.route("/lat_CH", methods=["GET", "POST"])
def lat_C():
    if request.method=='POST':
        
        url="http://localhost:8085/"
        return redirect(url)
    else:
        nickname=request.args.get('nickname')
        print(nickname)

        lat_CH(nickname)

        ex_name="랫 풀 다운"
        url_2 =f"http://localhost:8085/myInfo.do"

        return redirect(url_2)

# 스쿼트 트레이닝 라우더
@cross_origin
@app.route("/squat_TR", methods=["GET", "POST"])
def squat_T():
    if request.method=='POST':
        
        url="http://localhost:8085/flask"
        return redirect(url)
    else:
        print("들어왔따")
        nickname=request.args.get('nickname')
       
        squat_TR(nickname)

        ex_name="바벨 스쿼트"
        url_2 =f"http://localhost:8085/flaskFeedDetail.do?nickname={nickname}&ex_name={ex_name}"

        return redirect(url_2)


# 스쿼트 챌린지 라우더  
@cross_origin
@app.route("/squat_CH", methods=["GET", "POST"])
def squat_C():
    if request.method=='POST':
        
        url="http://localhost:8085/"
        return redirect(url)
    else:
        nickname=request.args.get('nickname')
    
        squat_CH(nickname)

        ex_name="바벨 스쿼트"
        url_2 =f"http://localhost:8085/myInfo.do?"

        return redirect(url_2)


# 바벨 컬 트레이닝 라우더
@cross_origin
@app.route("/cal_TR", methods=["GET", "POST"])
def cal_T():
    if request.method=='POST':
        print("바벨컬 포스트")
        url="http://localhost:8085/flask"
        return redirect(url)
    else:
        
        print("바벨 컬 입장~~~~~")
        nickname=request.args.get('nickname')
        print(nickname)

        cal_TR(nickname)

        ex_name="바벨 컬"
        url_2 =f"http://localhost:8085/flaskFeedDetail.do?nickname={nickname}&ex_name={ex_name}"

        return redirect(url_2)
  
# 바벨 컬 챌린지 라우더
@cross_origin
@app.route("/cal_CH", methods=["GET", "POST"])
def cal_C():
    if request.method=='POST':
        
        url="http://localhost:8085/"
        return redirect(url)
    else:
        nickname=request.args.get('nickname')
    

        cal_CH(nickname)

        ex_name="바벨 컬"
        url_2 =f"http://localhost:8085/myInfo.do?"

        return redirect(url_2)




# main함수의 시작을 의미 
# 비유하자면 자바의 main(String[] args)메소드와 같은 역할


if __name__ == '__main__':
    # 웹서버주소(host)와 포트번호(port)를 지정하여 웹 서버 실행
    app.run(host='127.0.0.1', port='5002') 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
